<a href="https://colab.research.google.com/github/Pabs-Gabriel/Applied-Data-Science-Capstone/blob/main/Module_1_Hands_on_Lab_Hands_on_Lab_Complete_the_Data_Collection_with_Web_Scraping_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install beautifulsoup4
!pip3 install requests

In [ ]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [ ]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    colunm_name = ' '.join(row.contents)

    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name


In [ ]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [ ]:
import requests

# Provided static URL
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# TASK 1: Perform HTTP GET request and store the response
response = requests.get(static_url)

# Optional: Print status code to check if request was successful
print("Status Code:", response.status_code)


Status Code: 200


In [ ]:
from bs4 import BeautifulSoup
import requests

# Fetch the page
url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"
response = requests.get(url)

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')



In [ ]:
# Use soup.title attribute

# Print the page title to verify the BeautifulSoup object
print("Page Title:", soup.title.string)


Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


In [ ]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
# Find all HTML tables with class 'wikitable'
tables = soup.find_all("table", class_="wikitable")

# Print the number of tables found
print("Number of tables found:", len(tables))


Number of tables found: 13


In [ ]:
# Select the third table (index 2 since Python is zero-indexed)
first_launch_table = tables[2]

# Print the HTML content of the third table
print(first_launch_table.prettify())


<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
 <tbody>
  <tr>
   <th scope="col">
    Flight No.
   </th>
   <th scope="col">
    Date and
    <br/>
    time (
    <a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">
     UTC
    </a>
    )
   </th>
   <th scope="col">
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">
     Version,
     <br/>
     Booster
    </a>
    <sup class="reference" id="cite_ref-booster_11-2">
     <a href="#cite_note-booster-11">
      <span class="cite-bracket">
       [
      </span>
      b
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </th>
   <th scope="col">
    Launch site
   </th>
   <th scope="col">
    Payload
    <sup class="reference" id="cite_ref-Dragon_12-2">
     <a href="#cite_note-Dragon-12">
      <span class="cite-bracket">
       [
      </span>
      c
      <span class="cite-bracket">
       ]
  

In [ ]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
column_names = []

# Find all <th> elements in the table header
th_elements = first_launch_table.find_all('th')

# Iterate through each <th> element
for th in th_elements:
    # Apply your provided extract_column_from_header() function to extract the column name
    name = extract_column_from_header(th)

    # Append only if the name is not None and not empty
    if name is not None and len(name) > 0:
        column_names.append(name)

# Print the extracted column names
print(column_names)


['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [ ]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


In [ ]:
# Initialize launch_dict with keys from extracted column_names
launch_dict = dict.fromkeys(column_names)

# Remove irrelevant column
del launch_dict['Date and time ( )']

# Initialize each key with an empty list to store data rows
for key in launch_dict.keys():
    launch_dict[key] = []

# Add additional columns that aren't in the header but you want to capture later
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

# Optional: print keys to verify
print("Columns in launch_dict:", list(launch_dict.keys()))


Columns in launch_dict: ['Flight No.', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Version Booster', 'Booster landing', 'Date', 'Time']


In [ ]:
extracted_row = 0

for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # get table rows
    for rows in table.find_all("tr"):
        # check if first <th> element contains flight number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
            else:
                flag = False
        else:
            flag = False

        # get all <td> elements in the row
        row = rows.find_all('td')

        # if flag is True, this row contains launch data
        if flag:
            extracted_row += 1

            # Append flight number
            launch_dict['Flight No.'].append(flight_number)

            # Extract date and time from the first cell
            datatimelist = date_time(row[0])

            # Append date (remove trailing comma)
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)

            # Append time
            time = datatimelist[1]
            launch_dict['Time'].append(time)

            # Extract booster version
            bv = booster_version(row[1])
            if not bv and row[1].a:
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)

            # Append Launch Site
            launch_site = row[2].a.string if row[2].a else row[2].text.strip()
            launch_dict['Launch site'].append(launch_site)

            # Append Payload
            payload = row[3].a.string if row[3].a else row[3].text.strip()
            launch_dict['Payload'].append(payload)

            # Append Payload mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)

            # Append Orbit
            orbit = row[5].a.string if row[5].a else row[5].text.strip()
            launch_dict['Orbit'].append(orbit)

            # Append Customer
            customer = row[6].a.string if row[6].a else row[6].text.strip()
            launch_dict['Customer'].append(customer)

            # Append Launch outcome
            launch_outcome = list(row[7].strings)[0].strip()
            launch_dict['Launch outcome'].append(launch_outcome)

            # Append Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)


In [ ]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })